### 📦 Install Required Python Packages

This cell installs the necessary Python libraries for audio processing, machine learning, and data balancing:

- `librosa`: A powerful library for audio and music analysis, useful for feature extraction from sound files.
- `pip --upgrade`: Ensures that `pip`, Python's package installer, is up to date to avoid installation issues.
- `xgboost`: A high-performance gradient boosting library used for classification and regression tasks.
- `imblearn`: Contains tools for handling imbalanced datasets, such as SMOTE for oversampling.
- `pydub`: A library for audio manipulation, including reading, converting, and slicing audio files.

```python
!pip install librosa
!pip install --upgrade pip
!pip install xgboost
!pip install imblearn
!pip install pydub


### 📚 Importing Required Libraries

This cell prepares the environment by importing libraries essential for:

- **Operating System Interaction**:
  - `os`: Allows file and directory manipulation, such as reading file names or navigating folders.

- **Numerical Computation**:
  - `numpy`: Provides support for efficient array operations and numerical calculations.

- **Audio Processing & Visualization**:
  - `librosa`: A specialized library for loading audio files and extracting audio features (like MFCCs).
  - `librosa.display`: Contains functions to visualize audio data, such as waveforms and spectrograms.

- **Machine Learning with Scikit-learn**:
  - `train_test_split`: Splits the dataset into training and testing sets.
  - `GridSearchCV`: Automates hyperparameter tuning using cross-validation.
  - `StratifiedKFold`: Ensures balanced class distribution in cross-validation folds.
  - `cross_validate`: Performs evaluation using multiple metrics and splits.
  - `StandardScaler`: Normalizes data by removing the mean and scaling to unit variance.
  - `classification_report`, `confusion_matrix`, `accuracy_score`: Provide metrics for evaluating model performance.

- **Model Training**:
  - `XGBClassifier`: The XGBoost classifier, known for its speed and accuracy in classification tasks.

- **Handling Imbalanced Data**:
  - `SMOTE` (Synthetic Minority Over-sampling Technique): Generates synthetic samples to balance class distributions in the training data.

- **Audio File Manipulation**:
  - `AudioSegment` (from `pydub`): Facilitates audio file conversion and editing (e.g., from MP3 to WAV).

> These imports set the foundation for building a machine learning pipeline focused on audio classification tasks.


In [1]:
!pip install librosa
!pip install --upgrade pip
!pip install xgboost
!pip install imblearn
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 7.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 21.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 37.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [imblearn]

In [2]:
pip install --upgrade scikit-learn imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade scikit-learn imbalanced-learn


In [4]:
import os
import numpy as np
import librosa
import librosa.display
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from pydub import AudioSegment

### 🎙️ `extract_features` Function Explanation

This function is designed to extract audio features from a given file for use in machine learning models, particularly for tasks like audio classification. Here's what each part does:

---

#### 🔧 Parameters:
- `file_path (str)`: Path to the audio file (.mp3 or .wav).
- `sample_rate (int, optional)`: Target sample rate for loading audio. If `None`, uses the default.
- `augment (bool)`: Whether to apply audio data augmentation.
- `is_real (bool)`: Whether to apply more aggressive augmentation (e.g., time stretching) for real recordings.

---

#### 🪄 Main Steps:

1. **Convert MP3 to WAV** (if needed):
   - If the input is an `.mp3` file, it is converted to a temporary `.wav` file using `pydub`.
   - This avoids decoding issues and allows `librosa` to process it.

2. **Load the Audio File**:
   - Uses `librosa.load()` to read the audio data and its sample rate.

3. **Clean Up**:
   - Deletes the temporary `.wav` file created during MP3 conversion.

4. **Data Augmentation (Optional)**:
   - Adds Gaussian noise.
   - Randomly shifts the pitch.
   - If `is_real` is `True`, also applies time stretching to simulate real-world variation.

5. **Feature Extraction**:
   - **MFCC (Mel-frequency cepstral coefficients)**: Captures timbral features of the sound.
   - **Spectral Contrast**: Measures the difference between peaks and valleys in the spectrum.
   - **ZCR (Zero Crossing Rate)**: Indicates signal noisiness or texture.

6. **Feature Aggregation**:
   - For each feature, both the **mean** and **standard deviation** are computed across time frames.
   - All these statistics are concatenated into a single feature vector (`np.ndarray`).

7. **Return**:
   - Returns the final 1D feature vector.
   - If any error occurs (e.g., decoding failure), the function prints an error and returns `None`.

---

> 🔁 This function enables consistent preprocessing and feature extraction from various audio file types, making it highly suitable for training and evaluating machine learning models.


In [5]:
def extract_features(file_path: str, sample_rate: int = None, augment: bool = False, is_real: bool = False) -> np.ndarray:
    try:
        # Handle .mp3 files by converting to .wav in a writable directory
        if file_path.endswith('.mp3'):
            try:
                audio = AudioSegment.from_mp3(file_path)
                temp_wav_path = os.path.join('/kaggle/working/', os.path.basename(file_path).replace('.mp3', '_temp.wav'))
                audio.export(temp_wav_path, format='wav')
                file_path = temp_wav_path
            except Exception as mp3_error:
                print(f"Skipping {file_path}: Failed to decode .mp3 file - {str(mp3_error)}")
                return None
        
        # Load audio file
        audio, sr = librosa.load(file_path, sr=sample_rate)
        
        # Clean up temporary file if created
        if file_path.endswith('_temp.wav'):
            os.remove(file_path)
        
        # Apply augmentation if enabled
        if augment:
            noise = np.random.normal(0, 0.015, audio.shape)
            audio = audio + noise
            audio = librosa.effects.pitch_shift(audio, sr=sr, n_steps=np.random.uniform(-4, 4))
            if is_real:
                audio = librosa.effects.time_stretch(audio, rate=np.random.uniform(0.6, 1.4))
        
        # Extract features (MFCC, spectral contrast, ZCR)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(audio)
        
        # Compute statistics (mean and std) for each feature
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)
        spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
        spectral_contrast_std = np.std(spectral_contrast, axis=1)
        zcr_mean = np.mean(zcr)
        zcr_std = np.std(zcr)
        
        # Concatenate all features into a single vector
        features = np.concatenate([
            mfcc_mean, mfcc_std,
            spectral_contrast_mean, spectral_contrast_std,
            [zcr_mean, zcr_std]
        ])
        
        return features
    
    except Exception as e:
        print(f"Error extracting features from {file_path}: {str(e)}")
        return None

### 📂 `load_dataset` Function Explanation

This function loads and processes audio data from a structured directory of real and fake audio samples, returning feature arrays and their corresponding labels for machine learning tasks.

---

#### 🔧 Parameters:
- `for_base_path (str)`: Root path where the audio folders (`for-2sec`, `for-norm`, etc.) are located.
- `augment (bool, default=True)`: Whether to apply data augmentation (noise, pitch shift, time stretch).
- `max_files_per_split (int, optional)`: If set, limits the number of audio files processed per data split (training/testing/validation).

---

#### 🪄 Workflow Overview:

1. **Helper Function – `get_audio_files(directory)`**:
   - Gathers all `.wav` and `.mp3` files from a given directory.
   - Respects the `max_files_per_split` limit if specified.

2. **Dataset Initialization**:
   - Initializes empty lists `X` (features) and `y` (labels).
   - Tracks the number of skipped files due to errors.

3. **Iterate Through Audio Folders**:
   - Loops through the subfolders: `for-2sec`, `for-norm`, `for-original`, `for-rerec`.
   - Each subfolder contains `training`, `testing`, and `validation` folders.

4. **Process Real and Fake Audio Files**:
   - For each split:
     - **Real audio files** are labeled with `0`.
     - **Fake audio files** are labeled with `1`.
     - For each file:
       - Features are extracted using `extract_features()`.
       - If `augment` is `True`, the augmented version is also added.

5. **Error Handling**:
   - Files that fail during processing are counted and reported.

6. **Return**:
   - Returns:
     - `X`: NumPy array of extracted feature vectors.
     - `y`: NumPy array of labels (0 for real, 1 for fake).
   - Prints the count of loaded samples and skipped files.

---

> 📌 **Directory Assumptions**:
> The function expects a directory structure like:
> ```
> for_base_path/
> └── for-original/
>     └── for-original/
>         ├── training/
>         │   ├── real/
>         │   └── fake/
>         └── testing/
>             ├── real/
>             └── fake/
> ```

> ✅ This function prepares labeled data for training or evaluating a binary classifier that distinguishes real from fake audio.


In [6]:
def load_dataset(for_base_path: str, augment: bool = True, max_files_per_split: int = None) -> tuple[np.ndarray, np.ndarray]:
    def get_audio_files(directory: str) -> list[str]:
        """Get list of .wav and .mp3 files in a directory."""
        try:
            files = [
                os.path.join(directory, f) for f in os.listdir(directory)
                if (f.endswith('.wav') or f.endswith('.mp3')) and os.path.isfile(os.path.join(directory, f))
            ]
            if max_files_per_split is not None:
                files = files[:max_files_per_split]
            return files
        except Exception as e:
            print(f"Error reading directory {directory}: {str(e)}")
            return []

    X, y = [], []
    skipped_files = 0
    
    for folder in ['for-2sec', 'for-norm', 'for-original', 'for-rerec']:
        subfolder_path = os.path.join(for_base_path, folder, folder)
        for split in ['training', 'testing', 'validation']:
            split_path = os.path.join(subfolder_path, split)
            if os.path.exists(split_path):
                real_path = os.path.join(split_path, 'real')
                if os.path.exists(real_path):
                    for file in get_audio_files(real_path):
                        features = extract_features(file, augment=augment, is_real=True)
                        if features is not None:
                            X.append(features)
                            y.append(0)
                            if augment:
                                aug_features = extract_features(file, augment=True, is_real=True)
                                if aug_features is not None:
                                    X.append(aug_features)
                                    y.append(0)
                                else:
                                    skipped_files += 1
                        else:
                            skipped_files += 1
                
                fake_path = os.path.join(split_path, 'fake')
                if os.path.exists(fake_path):
                    for file in get_audio_files(fake_path):
                        features = extract_features(file, augment=augment, is_real=False)
                        if features is not None:
                            X.append(features)
                            y.append(1)
                            if augment:
                                aug_features = extract_features(file, augment=True, is_real=False)
                                if aug_features is not None:
                                    X.append(aug_features)
                                    y.append(1)
                                else:
                                    skipped_files += 1
                        else:
                            skipped_files += 1
    
    if not X:
        raise ValueError("No valid audio files processed.")
    
    y = np.array(y)
    print(f"Loaded {len(X)} samples (real: {np.sum(y == 0)}, fake: {np.sum(y == 1)})")
    print(f"Skipped {skipped_files} files due to errors.")
    
    return np.array(X), y

### 🤖 `train_model` Function Explanation

This function trains an XGBoost classification model on the provided feature matrix `X` and label vector `y`, with optional hyperparameter tuning and class balancing strategies to improve performance on imbalanced datasets.

---

#### 🔧 Parameters:
- `X (np.ndarray)`: Feature matrix containing preprocessed audio features.
- `y (np.ndarray)`: Labels corresponding to the features (0 = real, 1 = fake).
- `tune_hyperparameters (bool, default=False)`: Whether to perform hyperparameter optimization using grid search.

---

### 🧠 Training Workflow:

#### 1. **Compute Class Weights**
- Determines the imbalance between real and fake classes.
- Adjusts `scale_pos_weight` for XGBoost accordingly, helping the model focus on the minority class.

#### 2. **Initialize the XGBoost Model**
- Uses predefined parameters (like `max_depth`, `learning_rate`, regularization, etc.).
- Integrates class weighting via `scale_pos_weight`.

#### 3. **Feature Scaling**
- Applies `StandardScaler` to normalize the feature values — an essential step before training many ML models.

#### 4. **Apply SMOTE (Synthetic Minority Oversampling Technique)**
- Balances the dataset by synthetically generating new samples of the minority class.
- Helps improve model generalization and reduce bias.

#### 5. **(Optional) Hyperparameter Tuning**
- If `tune_hyperparameters=True`, performs grid search using `GridSearchCV`:
  - Explores combinations of parameters (e.g., `n_estimators`, `max_depth`, `learning_rate`, etc.).
  - Evaluates using 5-fold cross-validation with the `f1_macro` score.

#### 6. **Model Training**
- If hyperparameter tuning is **not** enabled:
  - Splits data into training and validation sets.
  - Trains the model with early stopping on the validation set to prevent overfitting.

#### 7. **Cross-Validation Evaluation**
- Uses `StratifiedKFold` to maintain label distribution across folds.
- Reports metrics:
  - Accuracy
  - Precision (macro)
  - Recall (macro)
  - F1-score (macro)
  - ROC AUC
- Displays mean and standard deviation for each metric across folds.

#### 8. **Return Values**
- Returns:
  - The trained `XGBClassifier` model.
  - The `StandardScaler` used for preprocessing (important for transforming future input data).

---

> ✅ This function builds a robust classifier for detecting fake audio, handling class imbalance, applying best practices in preprocessing, and optionally tuning the model for optimal performance.


In [7]:
def train_model(X: np.ndarray, y: np.ndarray, tune_hyperparameters: bool = False) -> tuple[XGBClassifier, StandardScaler]:
    try:
        # Compute class weights for imbalanced dataset
        n_real = sum(y == 0)
        n_fake = sum(y == 1)
        class_weight_ratio = n_fake / n_real
        class_weights = {0: class_weight_ratio, 1: 1.0}
        
        # Initialize model
        model = XGBClassifier(
            n_estimators=80,
            max_depth=3,
            learning_rate=0.05,
            reg_alpha=0.5,
            reg_lambda=2.0,
            subsample=0.7,
            colsample_bytree=0.7,
            random_state=42,
            scale_pos_weight=class_weights[0]
        )
        scaler = StandardScaler()
        
        # Normalize features
        X_scaled = scaler.fit_transform(X)
        
        # Apply SMOTE to balance classes
        smote = SMOTE(random_state=42)
        X_scaled, y = smote.fit_resample(X_scaled, y)
        
        if tune_hyperparameters:
            param_grid = {
                'n_estimators': [60, 80, 100],
                'max_depth': [2, 3, 4],
                'learning_rate': [0.01, 0.05],
                'reg_alpha': [0.1, 0.5, 1.0],
                'reg_lambda': [1.0, 2.0, 3.0],
                'subsample': [0.6, 0.7],
                'colsample_bytree': [0.6, 0.7]
            }
            grid_search = GridSearchCV(
                model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1
            )
            grid_search.fit(X_scaled, y)
            model = grid_search.best_estimator_
            print(f"Best parameters: {grid_search.best_params_}")
        else:
            X_train, X_val, y_train, y_val = train_test_split(
                X_scaled, y, test_size=0.1, random_state=42, stratify=y
            )
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='logloss',
                early_stopping_rounds=15,
                verbose=True
            )
        
        # Perform cross-validation with stratified k-fold
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_validate(model, X_scaled, y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc'], return_train_score=False)
        print("Cross-validation results:")
        for metric in ['test_accuracy', 'test_precision_macro', 'test_recall_macro', 'test_f1_macro', 'test_roc_auc']:
            scores = cv_scores[metric]
            print(f"{metric}: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
        return model, scaler
    
    except Exception as e:
        print(f"Error during training: {str(e)}")
        raise

### 📊 `evaluate_model` Function Explanation

This function evaluates a trained XGBoost model on a test dataset, printing out key performance metrics.

---

#### 🔧 Parameters:
- `model (XGBClassifier)`: The trained classifier.
- `scaler (StandardScaler)`: The scaler used during training (to ensure consistent feature scaling).
- `X_test (np.ndarray)`: Feature matrix for the test set.
- `y_test (np.ndarray)`: True labels for the test set.

---

### 🧪 Evaluation Steps:

#### 1. **Feature Scaling**
- Transforms `X_test` using the same `StandardScaler` that was fitted on the training data.

#### 2. **Prediction**
- Computes class probabilities with `model.predict_proba()`.
- Converts probabilities to binary predictions using a threshold of `0.5`:
  - If probability ≥ 0.5 → class `1` (Fake).
  - Otherwise → class `0` (Real).

#### 3. **Metric Calculation**
- **Accuracy**: Overall correctness of predictions.
- **Classification Report**:
  - Precision, recall, and F1-score for each class (`Real`, `Fake`).
- **Confusion Matrix**:
  - Shows the number of true positives, false positives, true negatives, and false negatives.

#### 4. **Display Results**
- Prints all computed metrics clearly to summarize how well the model performs on unseen data.

---

> 📌 Use this function to validate the model’s effectiveness and identify any performance issues such as class imbalance or poor generalization.


In [8]:
def evaluate_model(model: XGBClassifier, scaler: StandardScaler, X_test: np.ndarray, y_test: np.ndarray) -> None:
    try:
        X_test_scaled = scaler.transform(X_test)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
        y_pred = (y_pred_proba >= 0.5).astype(int)
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, target_names=['Real', 'Fake'])
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        print(f"Test Accuracy: {accuracy:.4f}")
        print("Classification Report:\n", report)
        print("Confusion Matrix:\n", conf_matrix)
    
    except Exception as e:
        print(f"Error during evaluation: {str(e)}")
        raise

### 🔍 `predict_fake_audio` Function Explanation

This function takes a trained model and a new audio file, and predicts whether the audio is real or fake.

---

#### 🔧 Parameters:
- `model (XGBClassifier)`: The pre-trained classifier.
- `scaler (StandardScaler)`: The scaler fitted on training data, used to normalize new input features.
- `file_path (str)`: Path to the audio file (.wav or .mp3) to be predicted.

---

### 🧠 Prediction Workflow:

#### 1. **File Validation**
- Checks if the file exists at the specified path.
- If not found, raises a `FileNotFoundError`.

#### 2. **Feature Extraction**
- Calls `extract_features()` with:
  - `augment=False`: No data augmentation.
  - `is_real=False`: Treated as a test-time prediction (not training).
- If extraction fails, raises a `ValueError`.

#### 3. **Feature Scaling**
- Applies the same `StandardScaler` used during training to normalize the extracted features.

#### 4. **Model Prediction**
- Uses `model.predict_proba()` to get the probability of the audio being fake.
- Applies a threshold of `0.5`:
  - `proba ≥ 0.5` → Predicts **Fake audio**
  - `proba < 0.5` → Predicts **Real audio**

#### 5. **Returns**
- A human-readable prediction: either `"Real audio"` or `"Fake audio"`.

---

> 🎯 This function allows for real-time or batch inference on unseen audio data using a trained anti-spoofing model.


In [9]:
def predict_fake_audio(model: XGBClassifier, scaler: StandardScaler, file_path: str) -> str:
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        raise FileNotFoundError(f"File not found: {file_path}")
    
    features = extract_features(file_path, augment=False, is_real=False)
    if features is None:
        raise ValueError(f"Could not extract features from {file_path}")
    
    features_scaled = scaler.transform([features])
    proba = model.predict_proba(features_scaled)[0, 1]
    prediction = 1 if proba >= 0.5 else 0
    
    return "Real audio" if prediction == 0 else "Fake audio"

### 📁 Dataset Loading and Initialization

This code initializes the path to the dataset and loads a manageable subset of the data for testing.

---

#### 📌 Code Breakdown:

```python
# Define dataset path
for_base_path = '/kaggle/input/the-fake-or-real-dataset/'


In [10]:
# Define dataset path
for_base_path = '/kaggle/input/the-fake-or-real-dataset/'

# Load dataset with a limit for initial testing
print("Loading dataset...")
X, y = load_dataset(for_base_path, augment=True, max_files_per_split=1000)

Loading dataset...
Skipping /kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/fake/file17407.mp3: Failed to decode .mp3 file - Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 5.1.6-0+deb12u1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12 (Debian 12.2.0-14)
  configuration: --prefix=/usr --extra-version=0+deb12u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --

In [11]:
# Split data with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [12]:
# Train model with hyperparameter tuning
print("Training model...")
model, scaler = train_model(X_train, y_train, tune_hyperparameters=True)
        
# Evaluate model
print("Evaluating model...")
evaluate_model(model, scaler, X_test, y_test)

Training model...
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 1.0, 'subsample': 0.6}
Cross-validation results:
test_accuracy: 0.9558 (+/- 0.0052)
test_precision_macro: 0.9560 (+/- 0.0052)
test_recall_macro: 0.9558 (+/- 0.0052)
test_f1_macro: 0.9558 (+/- 0.0052)
test_roc_auc: 0.9906 (+/- 0.0012)
Evaluating model...
Test Accuracy: 0.9580
Classification Report:
               precision    recall  f1-score   support

        Real       0.97      0.95      0.96      3600
        Fake       0.95      0.97      0.96      3599

    accuracy                           0.96      7199
   macro avg       0.96      0.96      0.96      7199
weighted avg       0.96      0.96      0.96      7199

Confusion Matrix:
 [[3413  187]
 [ 115 3484]]


### 🧪 Example Prediction

This code snippet demonstrates how to use the trained model to make a prediction on a new audio file.

---

#### 📌 Code Breakdown:

```python
# Example prediction
new_audio_file = input("Enter the path to the audio file to check: ")


In [13]:
# Example prediction
new_audio_file = input("Enter the path to the audio file to check: ")
result = predict_fake_audio(model, scaler, new_audio_file)
print(f"Prediction for {new_audio_file}: {result}")

Enter the path to the audio file to check:  /kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/testing/fake/file1003.wav_16k.wav_norm.wav_mono.wav_silence.wav


Prediction for /kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/testing/fake/file1003.wav_16k.wav_norm.wav_mono.wav_silence.wav: Fake audio
